Second Problem - The ARX Model

    LOAD THE DATA

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

train_u = np.load("u_train.npy")
train_y = np.load("output_train.npy")
test_u = np.load("u_test.npy")

    Creation of the X matrix

In [3]:
import numpy as np

def X_matrix_creation(n,m,d,train_u,train_y):

    p = max(n, m + d)

    N = len(train_u)

    X_matrix = []

    def create_phi_matrix(k, n, m):

        y_theta = train_y[k-n:k]  
        y_theta = y_theta[::-1]  

        u_theta = train_u[k-d-m:k-d+1] 
        u_theta = u_theta[::-1] 
    
        phi = np.concatenate((y_theta, u_theta))

        return phi

    for k in range(p, N):  
        phi_matrix = create_phi_matrix(k, n, m)
        X_matrix.append(phi_matrix) 

    X_matrix = np.array(X_matrix)

    return X_matrix


    Creation of the Y matrix

In [4]:
def Y_matrix_creation(n,m,d,train_y):

    p = max(n,m+d)
    N = len(train_u)
    
    Y_matrix = []

    for k in range(p,N):
        y_p = np.array(train_y[k]) 
    
        Y_matrix = np.append(Y_matrix,y_p) 

    Y_matrix = np.array(Y_matrix).reshape(-1, 1) 

    return Y_matrix
    

    Y Predict Generator

In [5]:
import numpy as np

def create_phi_test(i, n, d, m, y_test, test_u):

    phi = []

    y_end_index = i - 1
    y_initial_index = i - n

    x_end_index = i - d
    x_initial_index = i - d - m

    for j in range(y_initial_index, y_end_index + 1):
        if j > 0:
            phi.append(y_test[j])
        else:
            phi.append(0)

    phi = phi[::-1] 

    for j in range (x_end_index + 1, x_initial_index,-1):
        if j >= 0:
            phi.append(test_u[j])
        else:
            phi.append(0)

    return phi

def y_predict_generator(model, X_matrix, n, m, d, test_u):
    
    N = len(test_u)
    p = max(n,m+d)
    theta_matrix = model.coef_

    y_test = [] 

    for i in range(0,N-1):

        if i < d:
            y_test.append(0)
        else:
            phi_test_vector = create_phi_test(i, n, d, m, y_test, test_u)

            phi_test_matrix = np.array(phi_test_vector).reshape(1, -1)

            y_pred = np.dot(phi_test_matrix, theta_matrix.T) + model.intercept_

            y_test.append(float(y_pred[0]))


    comparison_matrix = np.column_stack((train_y[-400:], y_test[-400:]))

    # Salvar a matriz comparativa em um arquivo .npy
    np.save("comparison_matrix.npy", comparison_matrix)

    return y_test[-400:]






    Best n,m,d parameters

In [ ]:
import numpy as np
from sklearn.linear_model import Ridge

def tune_ridge_model(X_matrix, Y_matrix):
   
    alpha_values = np.arange(1e-5, 1, 0.01) 
    best_sse = float('inf')
    best_alpha = None
    best_model = None

    for value in alpha_values:
        ridge_model = Ridge(alpha=value).fit(X_matrix, Y_matrix.ravel())
        y_pred = ridge_model.predict(X_matrix)

        sse = np.sum((Y_matrix.ravel() - y_pred) ** 2)

        if sse < best_sse:
            best_sse = sse
            best_alpha = value
            best_model = ridge_model

    return best_model, best_alpha, best_sse

best_sse = float('inf')
best_params = None
best_model = None

for n in range(1, 10):
    for m in range(1, 10):
        for d in range(1, 10):
   
            X_matrix = X_matrix_creation(n, m, d, train_u, train_y)
            Y_matrix = Y_matrix_creation(n, m, d, train_y)

            model, alpha, sse = tune_ridge_model(X_matrix, Y_matrix)

            if sse < best_sse:
                best_alpha = alpha
                best_sse = sse
                best_params = (n, m, d)
                best_model = model

print(f"Best parameters (n, m, d): {best_params}")
print(f"Best SSE: {best_sse}")


    1.4.7 Evaluation Model

In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

def evaluate_model(y_predicted,train_y,model,p,evaluation_df=None):

    flag = False

    if evaluation_df is None:
        best_sse = float('inf')
        evaluation_df = pd.DataFrame(columns=['Model', 'MSE', 'MAE', 'R²', 'SSE'])
        flag = True

    mse = mean_squared_error(train_y[-400:], y_predicted)
    mae = mean_absolute_error(train_y[-400:], y_predicted)
    r2 = r2_score(train_y[-400:], y_predicted)
    sse = np.sum((train_y[-400:] - y_predicted) ** 2)

    new_row = pd.DataFrame({
        'Model': [model],
        'MSE': [mse],
        'MAE': [mae],
        'R²': [r2],
        'SSE': [sse]
    })
    
    if flag:
        evaluation_df = new_row
    else:
        evaluation_df = pd.concat([evaluation_df, new_row], ignore_index=True)
        
    return evaluation_df


    1.4.8 Main Cycle

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import ElasticNet 
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.exceptions import ConvergenceWarning 

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

n = 9
m = 9
d = 6
p = max(n, m, d)

# X and Y matrix creation
X_matrix = X_matrix_creation(n, m, d, train_u, train_y)
Y_matrix = Y_matrix_creation(n, m, d, train_y)

# linear regression
linear_model = LinearRegression().fit(X_matrix, Y_matrix)
y_linear = y_predict_generator(linear_model, X_matrix, n, m, d, train_u)
df_linear = evaluate_model(y_linear, train_y, 'Lin. regr.', p, None)

# lasso
lasso_model = Lasso(alpha=0.000000000001).fit(X_matrix, Y_matrix)
y_lasso = y_predict_generator(lasso_model, X_matrix, n, m, d, train_u)
df_lasso = evaluate_model(y_lasso, train_y, 'Lasso', p, df_linear) 

# Ridge
ridge_model = Ridge(alpha=0.01)
ridge_model.fit(X_matrix, Y_matrix)
y_ridge = y_predict_generator(ridge_model, X_matrix, n, m, d, train_u)
df_ridge = evaluate_model(y_ridge, train_y, 'Ridge', p, df_lasso) 

# ElasticNet combines L1 and L2 regularization
elastic_net = ElasticNet(alpha=0.000001, l1_ratio=0.7)  
elastic_net.fit(X_matrix, Y_matrix)
y_elasticnet = y_predict_generator(elastic_net, X_matrix, n, m, d, train_u)
df_elasticnet = evaluate_model(y_elasticnet, train_y, 'Elasticnet', p, df_ridge) 

print(df_elasticnet)

    Plot

In [ ]:
N = len(train_u)
y_best_prediction = y_lasso

print(len(train_y[-400:]))
print(len(y_lasso))

train_y_sliced = train_y[-400:]
lasso_sliced = y_best_prediction

# Criar o gráfico
plt.figure(figsize=(10, 6))

# Plotar train_y
plt.plot(train_y_sliced, label='Train Y', color='blue')

# Plotar lasso
plt.plot(lasso_sliced, label='Lasso Prediction', color='red')

# Adicionar título e rótulos
plt.title('Comparison Between Train Y and Lasso Prediction')
plt.xlabel('Samples')
plt.ylabel('Values')

# Adicionar legenda
plt.legend()

# Mostrar o gráfico
plt.show()